In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb

In [2]:
df = pd.read_csv('../data/departures_250101_250430.csv')

summary = pd.DataFrame({
    'Dtype': df.dtypes,
    'Unique': df.nunique(),
    'NaN': df.isnull().sum(),
    'Zeros': (df == 0).sum()
})

display(df.shape)
summary

(32902, 35)

,Dtype,Unique,NaN,Zeros
type,object,1,0,0
status,object,3,0,0
departure.iataCode,object,1,0,0
departure.icaoCode,object,1,0,0
departure.terminal,float64,2,1335,0
departure.gate,object,52,3318,0
departure.delay,float64,215,3130,0
departure.scheduledTime,object,12522,0,0
departure.estimatedTime,object,15637,550,0
departure.actualTime,object,15259,3523,0


In [3]:
# List of IATA codes for airports in Schengen Area
# UPDATE IF NEEDED
schengen_airports = [
    'fra','waw','zrh','cdg','bgy','arn','brq','muc','ams','vie','bcn','edi',
    'ltn','rmf','hel','mxp','tfs','bru','ksc','agp','gla','lgw','dus','rix',
    'mad','dub','tuf','poz','lis','tia','gdn','sll','otp','alc','sof','bwe',
    'klu','pmi','nap','cph','beg','ein','nte','snu','fnc','blq','ath','stn',
    'bud','vlc','flr','kut','ayt','ncl','tsf','cta','rho','ema','rns','gro',
    'bri','ory','osr','lba','tbs','opo','spu','psa','crl','cia','ktt','psr',
    'lcy','lca','bio','lux','cag','lys','cgn','lpa','tat','gva','bsl','rkt',
    'nqz','kef','klx','lpl','prg','fue','vod','skg','bva','bhx','svq','pdl',
    'lbg','igs','krk','haj','got','bvc','mrs','lin','gyd','rmo','bah','var',
    'bfs','nce','ber','smv','ktw','vce','trs','her','inn','dla','mla','pqc',
    'pmo','str','ped','rmi','fao','cfu','rtm','bts','zad','sbz','hog','ala',
    'qzp','kbv','cvf','szg','kun','bqh','qrs','trn','adb','sir','asr','erf',
    'nue','zag','pfo','ndr','wro','qiu','grz','aey','lej','plq','ham','tsr',
    'peg','vrn','sma','qky','bll','sco','mct','pow','xry','tln','fae','bjz',
    'rze','mmx','ghv'
]

In [4]:
def add_weather(df_flights):
    # Data from Open-Meteo API
    df_weather = pd.read_csv('../data/weather_250101_250430.csv')
    
    # Column 'visibility_m' is all NaNs
    df_weather = df_weather.drop(columns=['visibility_m'])
    
    # Time in df_weather is in UTC but time in df_flights is in UTC+1/+2 (winter/summer)
    df_weather['time'] = pd.to_datetime(df_weather['time']).dt.tz_localize('UTC')
    df_weather['time'] = df_weather['time'].dt.tz_convert('Europe/Prague')
    df_weather['time'] = df_weather['time'].dt.tz_localize(None)
    
    df = pd.merge(df_flights, df_weather, left_on='join_time', right_on='time', how='left')
    
    df = df.drop(columns=['join_time', 'time'])
    return df

In [5]:
def add_traffic(df):
    # Departures
    df['hour_bucket'] = df['scheduled_time'].dt.round('h')
    
    departure_counts = df['hour_bucket'].value_counts()
    df['departure_traffic'] = df['hour_bucket'].map(departure_counts)
    
    df['departure_traffic'] = df['departure_traffic'] - 1
    
    # Arrivals
    df_arrivals = pd.read_csv('../data/arrivals_250101_250430.csv', low_memory=False)

    df_arrivals['scheduled_time'] = pd.to_datetime(df_arrivals['arrival.scheduledTime'])
    df_arrivals['hour_bucket'] = df_arrivals['scheduled_time'].dt.round('h')
    
    arrival_counts = df_arrivals['hour_bucket'].value_counts()
    
    df['arrival_traffic'] = df['hour_bucket'].map(arrival_counts).fillna(0)
    
    df = df.drop(columns=['hour_bucket'])
    
    return df

In [6]:
random_seed = 333

one_hot_enc = []

def get_dataset(one_hot=True, scale=True, scaler=StandardScaler()):
    global one_hot_enc
    
    df_flights = pd.read_csv('../data/departures_250101_250430.csv')
    
    # Drop irrelevant codeshared.* and arrival.* columns (except arrival.iataCode)
    cols_to_drop = [col for col in df_flights.columns if ((col.startswith('arrival.') and col != 'arrival.iataCode') or col.startswith('codeshared.'))]
    df_flights = df_flights.drop(columns=cols_to_drop)
    
    # Only keep rows with 'Active' status (drop 'Unknown' and 'Cancelled' flights)
    if 'status' in df_flights.columns:
        df_flights = df_flights[df_flights['status'] == 'active']
    
    if 'departure.estimatedTime' in df_flights.columns:
        df_flights = df_flights.drop(
            df_flights[df_flights['departure.estimatedTime'].isna()].index
        )
    
    # Let's make an actual departure time column (we will drop it later to prevent data leakage)
    df_flights['departure.actualTime2'] = df_flights['departure.actualTime'].fillna(df_flights['departure.estimatedTime'])
    df_flights = df_flights.drop(columns=['departure.actualTime', 'departure.estimatedTime'])
    df_flights = df_flights.rename(columns={'departure.actualTime2': 'departure.actualTime'})
    
    
    # NaNs in 'departure.delay' mean no delay so we can replace it with zeros.
    mask = (
        df_flights['departure.delay'].isna() &
        (df_flights['departure.scheduledTime'] >= df_flights['departure.actualTime'])
    )
    df_flights.loc[mask, 'departure.delay'] = 0
    
    cols_to_drop = [
        'departure.iataCode', 'departure.icaoCode', 'departure.gate',
        'departure.estimatedRunway', 'departure.actualRunway', 
        'airline.iataCode', 'airline.name', # maybe change iata/icao
        'status', 'type',
        'flight.number', 'flight.iataNumber', 'flight.icaoNumber'
    ]
    
    df_flights = df_flights.drop(columns=cols_to_drop)
    
    df_flights = df_flights.rename(columns={
        'departure.terminal': 'terminal',
        'departure.delay': 'delay',
        'departure.scheduledTime': 'scheduled_time',
        'airline.icaoCode': 'airline',
        'departure.actualTime': 'actual_time',
        'arrival.iataCode': 'destination_airport',
    })

    # convert to datetime64
    df_flights['scheduled_time'] = pd.to_datetime(df_flights['scheduled_time'])
    df_flights['actual_time'] = pd.to_datetime(df_flights['actual_time'])

    # add a join time to match weather
    df_flights['join_time'] = df_flights['scheduled_time'].dt.round('h')

    # add weather
    df_weather = add_weather(df_flights)
    
    # add traffic
    df = add_traffic(df_weather)

    df = add_traffic(df_weather)

    # drop actual time
    df.drop(columns=['actual_time'], inplace=True)

    # Convert scheduled_time to columns that are relevant for ML
    df['day_of_week'] = df['scheduled_time'].dt.weekday
    df['day_in_month'] = df['scheduled_time'].dt.day
    df['hour'] = df['scheduled_time'].dt.round('h').dt.hour

    df.drop(columns=['scheduled_time'], inplace=True)

    # cyclical features (maybe comment out)
    
    # hours
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df.drop(columns=['hour'],inplace=True)
    
    # day of week
    df['weekday_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df.drop(columns=['day_of_week'],inplace=True)
    
    # drop rows with NaNs (there is only 2, easier to drop than to ampute)
    df = df[df['airline'].isna() == 0]
    df = df[df['temp_c'].isna() == 0]

    # fill NaN with terminal number according to whether destination airport is in Schengen Area or not
    df['terminal'] = df['terminal'].fillna(df['destination_airport'].isin(schengen_airports).map({True: 2, False: 1}))

    categorical = ['terminal', 'destination_airport', 'airline']
    numerical = ['temp_c', 'precip_mm', 'snow_cm', 'wind_kph', 'departure_traffic', 'arrival_traffic', 'day_in_month']
    
    # cat codes
    if not one_hot:
        for col in categorical:
            category = pd.api.types.CategoricalDtype(categories=df[col].dropna().unique(), ordered=False)
            df[col] = df[col].astype(category).cat.codes

    Xtrain, Xrest, ytrain, yrest = train_test_split(df.drop(columns=['delay']), df['delay'], test_size=0.4, random_state=random_seed)
    Xval, Xtest, yval, ytest = train_test_split(Xrest, yrest, test_size=0.5, random_state=random_seed)

    if one_hot:
        one_hot_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        
        # fit only on Xtrain and transform to OHE
        train_ohe = one_hot_enc.fit_transform(Xtrain[categorical]).astype(int)
        val_ohe   = one_hot_enc.transform(Xval[categorical]).astype(int)
        test_ohe  = one_hot_enc.transform(Xtest[categorical]).astype(int)

        ohe_cols = one_hot_enc.get_feature_names_out(categorical)

        # drop old cols
        Xtrain = Xtrain.drop(columns=categorical)
        Xval   = Xval.drop(columns=categorical)
        Xtest  = Xtest.drop(columns=categorical)

        # concat new cols back
        Xtrain = pd.concat([Xtrain.reset_index(drop=True),
                            pd.DataFrame(train_ohe, columns=ohe_cols)], axis=1)
        
        Xval = pd.concat([Xval.reset_index(drop=True),
                          pd.DataFrame(val_ohe, columns=ohe_cols)], axis=1)
        
        Xtest = pd.concat([Xtest.reset_index(drop=True),
                           pd.DataFrame(test_ohe, columns=ohe_cols)], axis=1)
        
    if scale:
        Xtrain[numerical] = scaler.fit_transform(Xtrain[numerical])
        Xval[numerical] = scaler.transform(Xval[numerical])
        Xtest[numerical] = scaler.transform(Xtest[numerical])
    
    return Xtrain, Xval, Xtest, ytrain, yval, ytest

Xtrain, Xval, Xtest, ytrain, yval, ytest = get_dataset(one_hot=True)